### **Create the image folders for the corresponding classes**

In [8]:
import cv2
import glob
import os

#import torch
#Path of dataset and new dataset,change these two variables if needed 
path="C:/Users/vshar/OneDrive/Desktop/Finger/fingers"
new_path = "C:/Users/vshar/OneDrive/Desktop/Assignment3"

subDirectories = os.listdir(path) 

#Image Dataset holder
images = []
imageList= []

#from csv import writer
 

hands=['L','R'] 
labels1=[]
labels2={}

for hand in hands:
    for i in range(0,6):
        labels1.append(str(i)+hand) 
        labels2[str(i)+hand]=str(i)+hand

counterImagesProcessed=0
if os.path.exists(new_path)==False:
    os.mkdir(new_path)
    for folder in subDirectories:
      os.mkdir(new_path+"/"+folder)
      for category in labels2:
        os.mkdir(new_path+"/"+folder+"/"+category)
    #newSubDirectories = os.listdir(new_path) 
    for folder in subDirectories:
        currentPath=path+"/" +folder
        images = os.listdir(currentPath)
        for image in images:
            newCurrentPath=new_path+"/"+folder+"/"+image[-6]+image[-5]+"/"+image
            cv2.imwrite(newCurrentPath,cv2.imread(currentPath+"/"+image))
            counterImagesProcessed=counterImagesProcessed+1

else:
    print("Dataset Alreay Exists")

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/fingers'

### **Import Libraries**

In [4]:
from math import ceil
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as td
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import image
from matplotlib import pyplot
import matplotlib.pyplot as plt
import time
from torchvision import models

### **Set device to GPU or CPU. Also set dataset label map**

In [5]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
labels_map = {
      0: "0-Left",
      1: "0-Right",
      2: "1-Left ",
      3: "1-Right ",
      4: "2-Left ",
      5: "2-Right ",
      6: "3-Left ",
      7: "3-Right ",
      8: "4-Left ",
      9: "4-Right",
      10: "5-Left ",
      11: "5-Right ",
}

### **Data Loader**

In [6]:
def data_loader(data_dir_input,batch_sizeGiven,input_size,test_split,val_split,flag=0):
# Define dataset directory and transforms
  data_dir = data_dir_input #
  test_transform=train_transform = transforms.Compose([
      transforms.Resize(input_size, interpolation=transforms.InterpolationMode.BILINEAR),
      transforms.RandomRotation(50),
      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
      transforms.RandomPerspective(distortion_scale=0.2, p=0.2),
      transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
      transforms.ToTensor(),           
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])


# Define train, validation, and test dataset
  train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train/'), transform=train_transform)
  data_directory_test = data_dir+"/"+"test"+"/"
  test_dataset = datasets.ImageFolder(data_directory_test, transform=test_transform)
  train_loader = DataLoader(train_dataset, batch_size=batch_sizeGiven, shuffle=True, drop_last=False, num_workers=0,pin_memory=True)
  val_size = int(ceil((len(test_dataset)*val_split) / (test_split+val_split)))
  test_size = len(test_dataset) - val_size
  test_dataset, val_dataset = td.random_split(test_dataset, [test_size, val_size])


  test_loader = DataLoader(test_dataset, batch_size=batch_sizeGiven, shuffle=True, drop_last=False, num_workers=0,pin_memory=True) #shuffle 
  val_loader = DataLoader(val_dataset, batch_size=batch_sizeGiven, shuffle=True, drop_last=False, num_workers=0,pin_memory=True) # shuffle working
  if flag==1:
    print("Train Datset Size Before Split",len(train_dataset))
    print("Test Datset Size Before Split",len(test_dataset))
    print("################################################")
    print("Train Datset Size After Split",len(train_dataset))
    print("Test Datset Size After Split",len(test_dataset))
    print("Validation Datset Size After Split",len(val_dataset))

    figure = plt.figure(figsize=(8, 8))
    cols, rows = 3, 3
    for i in range(1, cols * rows + 1):
      sample_idx = torch.randint(len(train_dataset), size=(1,)).item()
      img, label = train_dataset[sample_idx]
      img=img.view(128,128,3)
      figure.add_subplot(rows, cols, i)
      plt.title(labels_map[label])
      plt.axis("off")
      plt.imshow(img.squeeze())
    print("\n\n\n################################################\n\n\n")
    print("Dataset After Pre-Processing")
    plt.show()
    return  



  return train_loader,test_loader,val_loader
 

### **Dataset Image**

In [ ]:
BeforeImage = image.imread(new_path+"/test/0L/00ab429e-7edd-4cf7-b6a6-25eec65d5cda_0L.png")
plt.figure()
plt.title("0 Left (Before Preprocessing)")
plt.imshow(BeforeImage)

### **Preprocessed Images**

In [ ]:
def Display(data_dir_input,batch_sizeGiven,input_size,test_split,val_split,flag=1):
  data_loader(data_dir_input,batch_sizeGiven,input_size,test_split,val_split,1)
Display("C:/Users/vshar/OneDrive/Desktop/Finger/fingers",32, (128, 128) ,0.2, 0.1,1 )

### **Training Function**

In [ ]:
def train(num_epochsGiven, model, train_loader, criterion, optimizer,val_loader):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print("Device: {}".format(device))
  model.to(device)
  Accuracy=[]
  validAccuracy=[]
  Loss = []
  num_epochs = num_epochsGiven
  total_steps = len(train_loader)
  t1 = time.time()
  total,correct,loss=0,0,0
  for epoch in range(num_epochs):
      stepAcc=[]
      stepLoss=[]    
      for i, data in enumerate(train_loader):
          images, labels = data[0].to(device), data[1].to(device)
          # Forward pass
          outputs = model(images)
          loss = criterion(outputs, labels)
          # Backprop and optimisation
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          # Train accuracy
          total = labels.size(0)
          _,predicted = torch.max(outputs.data, 1)
          correct = (predicted == labels).sum().item()
          stepAcc.append((correct / total) * 100)
          stepLoss.append(loss.item())
          if (i + 1) % 100 == 0:
              print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                     (correct / total) * 100))
      correct_v = 0
      total_v = 0
      for dataVal in val_loader:
          images_v, labels_v = dataVal[0].to(device), dataVal[1].to(device)
          outputs = model(images_v)
          _, predicted = torch.max(outputs.data, 1)
          correct_v += (predicted == labels_v).sum().item()
          total_v += labels_v.size(0)        
      Accuracy.append(sum(stepAcc)/len(stepAcc))
      validAccuracy.append((correct_v / total_v) * 100)
      Loss.append(sum(stepLoss)/len(stepLoss))
  print("######## Training Finished in {} seconds ###########".format(time.time()-t1))
  return Loss,Accuracy,model,validAccuracy

### **Testing of Model using different metrics**

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report
def test(model, device, test_loader):
  model.eval() 
  y_truth=[]
  y_predicted=[]
  cm=[]
  with torch.no_grad(): 
      correct = 0
      total = 0
      for data in test_loader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          #print(predicted,labels)
          correct += (predicted == labels).sum().item()
          y_truth+=labels.cpu().numpy().tolist()
          y_predicted+=predicted.cpu().numpy().tolist()
      print('Test Accuracy of the model on the {} test images: {} %'.format(total, (correct / total) * 100))
      cm= confusion_matrix(y_truth,y_predicted)
  print(classification_report(y_truth,y_predicted))
  return cm


### **Model (ResNet18)(ImageNet Version1 )**

In [ ]:
MODELS3=[]
hyper_parameters3 = []
train_acc_hyper_paramaters3=[]
train_acc_valid_hyper_paramaters3=[]
train_loss_hyper_paramaters3=[]

stringCrit="Cross Entropy Loss"
criterion=nn.CrossEntropyLoss()
batch_sizes=[32,64]
learning_rates=[0.001,.0001]
epochs=5


for learning_rate in learning_rates:
  for batch_size in batch_sizes:
    for optimizer_count in range(2):
      model3 = models.resnet18(weights="ResNet18_Weights.IMAGENET1K_V1",progress=False)
      model3.fc = nn.Linear(512, 12)
      stringOPTM=""
      if optimizer_count%2==0:
       stringOPTM="Adam" 
       optimizer = torch.optim.Adam(model3.parameters(), lr=learning_rate)
      else:
        stringOPTM="SGD"
        optimizer = torch.optim.SGD(model3.parameters(), lr=learning_rate, momentum=0.9)
      hyper_parameters3.append(["Learning Rate: "+str(learning_rate)+" Batch Size: "+str(batch_size),stringCrit,stringOPTM])
      print("Learning Rate: "+str(learning_rate)+" Batch Size: "+str(batch_size)," Loss function: ",stringCrit," Optimizer: ",stringOPTM)
      #optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate)
      train_loader, test_loader, val_loader = data_loader("/content/Assignment3",batch_size, (224, 224) ,0.2, 0.1 )
      tempLoss,tempAccuracy,tempModel,tempValidAccuracy = train(epochs,model3,train_loader,criterion,optimizer,val_loader)
      train_loss_hyper_paramaters3.append(tempLoss)
      train_acc_hyper_paramaters3.append(tempAccuracy)
      train_acc_valid_hyper_paramaters3.append(tempValidAccuracy)
      cmReturned = test(model3,"cuda",test_loader)
      if cmReturned is not None:
          fig, ax = plt.subplots(figsize=(7, 7))
          ConfusionMatrixDisplay(cmReturned).plot(ax=ax,cmap='Blues', xticks_rotation='vertical', values_format='d')
          plt.show()
      torch.save(tempModel,'/content/drive/MyDrive/Models Resnet TF/'+"_".join(["Learning Rate: "+str(learning_rate)+" Batch Size: "+str(batch_size),stringCrit,stringOPTM])+".pth")
      torch.save(tempModel,'/content/drive/MyDrive/Models Resnet TF/'+"_".join(["Learning Rate: "+str(learning_rate)+" Batch Size: "+str(batch_size),stringCrit,stringOPTM])+".pt")     
      MODELS3.append(tempModel)
  

### **Accuracy Plot**

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


from numpy.core.multiarray import dtype
####### plot the the training accuracy here #########
num_epochs=5
epochs = [i for i in range(num_epochs)]
plt.figure(figsize=(15, 15))
plt.xlabel('epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy plot')

#train_loss_hyper_paramaters.numpy()
#print(train_loss_hyper_paramaters["lr0.01"])

for i in range(len(hyper_parameters3)):
    plt.plot(train_acc_hyper_paramaters3[i],label=hyper_parameters3[i])
plt.legend()

### **Loss Plot**

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


from numpy.core.multiarray import dtype
####### plot the the training loss here #########
num_epochs=5
epochs = [i for i in range(num_epochs)]
plt.figure(figsize=(15, 15))
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.title('Loss plot ResNet18')

#train_loss_hyper_paramaters.numpy()
#print(train_loss_hyper_paramaters["lr0.01"])

for i in range(len(hyper_parameters3)):
    plt.plot(train_loss_hyper_paramaters3[i],label=hyper_parameters3[i])
plt.legend()

### **Validation Accuracy**

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


from numpy.core.multiarray import dtype
####### plot the the training loss here #########
num_epochs=5
epochs = [i for i in range(num_epochs)]
plt.figure(figsize=(15, 15))
plt.xlabel('epoch')
plt.ylabel('Validation Accuracy')
plt.title('Validation plot ResNet18')

#train_loss_hyper_paramaters.numpy()
#print(train_loss_hyper_paramaters["lr0.01"])

for i in range(len(hyper_parameters3)):
    plt.plot(train_acc_valid_hyper_paramaters3[i],label=hyper_parameters3[i])
plt.legend()

### **Input image to the trained models**

In [ ]:
TempTransformer = transforms.Compose([
      transforms.Resize((128,128), interpolation=transforms.InterpolationMode.BILINEAR),
      transforms.RandomRotation(50),
      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
      transforms.RandomPerspective(distortion_scale=0.2, p=0.2),
      transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
      transforms.ToTensor(),           
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])


from PIL import Image
imageInput = Image.open(new_path+"/train/5L/009850f5-6bf5-445d-88dd-25f6ccbe538b_5L.png")
input_data = TempTransformer(imageInput)
input_data = input_data.to(device)
input_data = input_data.unsqueeze(0)
i=0
for tempModelList in MODELS3:
  tempModelList.eval()
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  tempModelList.to(device)
  print(" ".join(hyper_parameters3[i]))
  with torch.no_grad():
    output = tempModelList(input_data)

  out=output.tolist()[0]
  print(labels_map[out.index(max(out))])
  i=i+1

### **Unzip the dataset,downloaded from kaggle on google colab**

In [ ]:
!unzip archive.zip

### **Create the image folders for the corresponding classes**

In [ ]:
import cv2
import glob
import os

#import torch
#Path of dataset and new dataset,change these two variables if needed 
path="/content/fingers"
new_path = "/content/Assignment3"

subDirectories = os.listdir(path) 

#Image Dataset holder
images = []
imageList= []

#from csv import writer
 

hands=['L','R'] 
labels1=[]
labels2={}

for hand in hands:
    for i in range(0,6):
        labels1.append(str(i)+hand) 
        labels2[str(i)+hand]=str(i)+hand

counterImagesProcessed=0
if os.path.exists(new_path)==False:
    os.mkdir(new_path)
    for folder in subDirectories:
      os.mkdir(new_path+"/"+folder)
      for category in labels2:
        os.mkdir(new_path+"/"+folder+"/"+category)
    #newSubDirectories = os.listdir(new_path) 
    for folder in subDirectories:
        currentPath=path+"/" +folder
        images = os.listdir(currentPath)
        for image in images:
            newCurrentPath=new_path+"/"+folder+"/"+image[-6]+image[-5]+"/"+image
            cv2.imwrite(newCurrentPath,cv2.imread(currentPath+"/"+image))
            counterImagesProcessed=counterImagesProcessed+1

else:
    print("Dataset Alreay Exists")

### **Import Libraries**

In [ ]:
from math import ceil
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as td
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import image
from matplotlib import pyplot
import matplotlib.pyplot as plt
import time
from torchvision import models

### **Set device to GPU or CPU. Also set dataset label map**

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
labels_map = {
      0: "0-Left",
      1: "0-Right",
      2: "1-Left ",
      3: "1-Right ",
      4: "2-Left ",
      5: "2-Right ",
      6: "3-Left ",
      7: "3-Right ",
      8: "4-Left ",
      9: "4-Right",
      10: "5-Left ",
      11: "5-Right ",
}

### **Data Loader**

In [ ]:
def data_loader(data_dir_input,batch_sizeGiven,input_size,test_split,val_split,flag=0):
# Define dataset directory and transforms
  data_dir = data_dir_input #
  test_transform=train_transform = transforms.Compose([
      transforms.Resize(input_size, interpolation=transforms.InterpolationMode.BILINEAR),
      transforms.RandomRotation(50),
      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
      transforms.RandomPerspective(distortion_scale=0.2, p=0.2),
      transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
      transforms.ToTensor(),           
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])


# Define train, validation, and test dataset
  train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train/'), transform=train_transform)
  data_directory_test = data_dir+"/"+"test"+"/"
  test_dataset = datasets.ImageFolder(data_directory_test, transform=test_transform)
  train_loader = DataLoader(train_dataset, batch_size=batch_sizeGiven, shuffle=True, drop_last=False, num_workers=0,pin_memory=True)
  val_size = int(ceil((len(test_dataset)*val_split) / (test_split+val_split)))
  test_size = len(test_dataset) - val_size
  test_dataset, val_dataset = td.random_split(test_dataset, [test_size, val_size])


  test_loader = DataLoader(test_dataset, batch_size=batch_sizeGiven, shuffle=True, drop_last=False, num_workers=0,pin_memory=True) #shuffle 
  val_loader = DataLoader(val_dataset, batch_size=batch_sizeGiven, shuffle=True, drop_last=False, num_workers=0,pin_memory=True) # shuffle working
  if flag==1:
    print("Train Datset Size Before Split",len(train_dataset))
    print("Test Datset Size Before Split",len(test_dataset))
    print("################################################")
    print("Train Datset Size After Split",len(train_dataset))
    print("Test Datset Size After Split",len(test_dataset))
    print("Validation Datset Size After Split",len(val_dataset))

    figure = plt.figure(figsize=(8, 8))
    cols, rows = 3, 3
    for i in range(1, cols * rows + 1):
      sample_idx = torch.randint(len(train_dataset), size=(1,)).item()
      img, label = train_dataset[sample_idx]
      img=img.view(128,128,3)
      figure.add_subplot(rows, cols, i)
      plt.title(labels_map[label])
      plt.axis("off")
      plt.imshow(img.squeeze())
    print("\n\n\n################################################\n\n\n")
    print("Dataset After Pre-Processing")
    plt.show()
    return  



  return train_loader,test_loader,val_loader
 

### **Dataset Image**

In [ ]:
BeforeImage = image.imread(new_path+"/test/0L/00ab429e-7edd-4cf7-b6a6-25eec65d5cda_0L.png")
plt.figure()
plt.title("0 Left (Before Preprocessing)")
plt.imshow(BeforeImage)

### **Preprocessed Images**

In [ ]:
def Display(data_dir_input,batch_sizeGiven,input_size,test_split,val_split,flag=1):
  data_loader(data_dir_input,batch_sizeGiven,input_size,test_split,val_split,1)
Display("/content/Assignment3",32, (128, 128) ,0.2, 0.1,1 )

### **Training Function**

In [ ]:
def train(num_epochsGiven, model, train_loader, criterion, optimizer,val_loader):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print("Device: {}".format(device))
  model.to(device)
  Accuracy=[]
  validAccuracy=[]
  Loss = []
  num_epochs = num_epochsGiven
  total_steps = len(train_loader)
  t1 = time.time()
  total,correct,loss=0,0,0
  for epoch in range(num_epochs):
      stepAcc=[]
      stepLoss=[]    
      for i, data in enumerate(train_loader):
          images, labels = data[0].to(device), data[1].to(device)
          # Forward pass
          outputs = model(images)
          loss = criterion(outputs, labels)
          # Backprop and optimisation
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          # Train accuracy
          total = labels.size(0)
          _,predicted = torch.max(outputs.data, 1)
          correct = (predicted == labels).sum().item()
          stepAcc.append((correct / total) * 100)
          stepLoss.append(loss.item())
          if (i + 1) % 100 == 0:
              print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                     (correct / total) * 100))
      correct_v = 0
      total_v = 0
      for dataVal in val_loader:
          images_v, labels_v = dataVal[0].to(device), dataVal[1].to(device)
          outputs = model(images_v)
          _, predicted = torch.max(outputs.data, 1)
          correct_v += (predicted == labels_v).sum().item()
          total_v += labels_v.size(0)        
      Accuracy.append(sum(stepAcc)/len(stepAcc))
      validAccuracy.append((correct_v / total_v) * 100)
      Loss.append(sum(stepLoss)/len(stepLoss))
  print("######## Training Finished in {} seconds ###########".format(time.time()-t1))
  return Loss,Accuracy,model,validAccuracy

### **Testing of Model using different metrics**

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report
def test(model, device, test_loader):
  model.eval() 
  y_truth=[]
  y_predicted=[]
  cm=[]
  with torch.no_grad(): 
      correct = 0
      total = 0
      for data in test_loader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          #print(predicted,labels)
          correct += (predicted == labels).sum().item()
          y_truth+=labels.cpu().numpy().tolist()
          y_predicted+=predicted.cpu().numpy().tolist()
      print('Test Accuracy of the model on the {} test images: {} %'.format(total, (correct / total) * 100))
      cm= confusion_matrix(y_truth,y_predicted)
  print(classification_report(y_truth,y_predicted))
  return cm


### **Model (ResNet18)(ImageNet Version1 )**

In [ ]:
MODELS3=[]
hyper_parameters3 = []
train_acc_hyper_paramaters3=[]
train_acc_valid_hyper_paramaters3=[]
train_loss_hyper_paramaters3=[]

stringCrit="Cross Entropy Loss"
criterion=nn.CrossEntropyLoss()
batch_sizes=[32,64]
learning_rates=[0.001,.0001]
epochs=5


for learning_rate in learning_rates:
  for batch_size in batch_sizes:
    for optimizer_count in range(2):
      model3 = models.resnet18(weights="ResNet18_Weights.IMAGENET1K_V1",progress=False)
      model3.fc = nn.Linear(512, 12)
      stringOPTM=""
      if optimizer_count%2==0:
       stringOPTM="Adam" 
       optimizer = torch.optim.Adam(model3.parameters(), lr=learning_rate)
      else:
        stringOPTM="SGD"
        optimizer = torch.optim.SGD(model3.parameters(), lr=learning_rate, momentum=0.9)
      hyper_parameters3.append(["Learning Rate: "+str(learning_rate)+" Batch Size: "+str(batch_size),stringCrit,stringOPTM])
      print("Learning Rate: "+str(learning_rate)+" Batch Size: "+str(batch_size)," Loss function: ",stringCrit," Optimizer: ",stringOPTM)
      #optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate)
      train_loader, test_loader, val_loader = data_loader("/content/Assignment3",batch_size, (224, 224) ,0.2, 0.1 )
      tempLoss,tempAccuracy,tempModel,tempValidAccuracy = train(epochs,model3,train_loader,criterion,optimizer,val_loader)
      train_loss_hyper_paramaters3.append(tempLoss)
      train_acc_hyper_paramaters3.append(tempAccuracy)
      train_acc_valid_hyper_paramaters3.append(tempValidAccuracy)
      cmReturned = test(model3,"cuda",test_loader)
      if cmReturned is not None:
          fig, ax = plt.subplots(figsize=(7, 7))
          ConfusionMatrixDisplay(cmReturned).plot(ax=ax,cmap='Blues', xticks_rotation='vertical', values_format='d')
          plt.show()
      torch.save(tempModel,'/content/drive/MyDrive/Models Resnet TF/'+"_".join(["Learning Rate: "+str(learning_rate)+" Batch Size: "+str(batch_size),stringCrit,stringOPTM])+".pth")
      torch.save(tempModel,'/content/drive/MyDrive/Models Resnet TF/'+"_".join(["Learning Rate: "+str(learning_rate)+" Batch Size: "+str(batch_size),stringCrit,stringOPTM])+".pt")     
      MODELS3.append(tempModel)
  

### **Accuracy Plot**

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


from numpy.core.multiarray import dtype
####### plot the the training accuracy here #########
num_epochs=5
epochs = [i for i in range(num_epochs)]
plt.figure(figsize=(15, 15))
plt.xlabel('epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy plot')

#train_loss_hyper_paramaters.numpy()
#print(train_loss_hyper_paramaters["lr0.01"])

for i in range(len(hyper_parameters3)):
    plt.plot(train_acc_hyper_paramaters3[i],label=hyper_parameters3[i])
plt.legend()

### **Loss Plot**

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


from numpy.core.multiarray import dtype
####### plot the the training loss here #########
num_epochs=5
epochs = [i for i in range(num_epochs)]
plt.figure(figsize=(15, 15))
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.title('Loss plot ResNet18')

#train_loss_hyper_paramaters.numpy()
#print(train_loss_hyper_paramaters["lr0.01"])

for i in range(len(hyper_parameters3)):
    plt.plot(train_loss_hyper_paramaters3[i],label=hyper_parameters3[i])
plt.legend()

### **Validation Accuracy**

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


from numpy.core.multiarray import dtype
####### plot the the training loss here #########
num_epochs=5
epochs = [i for i in range(num_epochs)]
plt.figure(figsize=(15, 15))
plt.xlabel('epoch')
plt.ylabel('Validation Accuracy')
plt.title('Validation plot ResNet18')

#train_loss_hyper_paramaters.numpy()
#print(train_loss_hyper_paramaters["lr0.01"])

for i in range(len(hyper_parameters3)):
    plt.plot(train_acc_valid_hyper_paramaters3[i],label=hyper_parameters3[i])
plt.legend()

### **Input image to the trained models**

In [ ]:
TempTransformer = transforms.Compose([
      transforms.Resize((128,128), interpolation=transforms.InterpolationMode.BILINEAR),
      transforms.RandomRotation(50),
      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
      transforms.RandomPerspective(distortion_scale=0.2, p=0.2),
      transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
      transforms.ToTensor(),           
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])


from PIL import Image
imageInput = Image.open(new_path+"/train/5L/009850f5-6bf5-445d-88dd-25f6ccbe538b_5L.png")
input_data = TempTransformer(imageInput)
input_data = input_data.to(device)
input_data = input_data.unsqueeze(0)
i=0
for tempModelList in MODELS3:
  tempModelList.eval()
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  tempModelList.to(device)
  print(" ".join(hyper_parameters3[i]))
  with torch.no_grad():
    output = tempModelList(input_data)

  out=output.tolist()[0]
  print(labels_map[out.index(max(out))])
  i=i+1